# Supervised Learning Methods for identifying Dolphin Attacks

## 1. Import Data

In [183]:
import os
import librosa
import librosa.display
import glob
import numpy as np
from scipy.io import wavfile
from matplotlib import pyplot as plt
%matplotlib inline

In [184]:
#Return audio Features from input file
def getAudioFeatures(file):
    sample, sr = librosa.load(file)
    mfcc = librosa.feature.mfcc(y=sample, sr=sr, n_mfcc=4)
    return mfcc
    

In [185]:
#Define paths
GOOGLE_ATTACKS = ['./samples/google_attacks/'+x for x in os.listdir('./samples/google_attacks/')]
GOOGLE_NORMAL = ['./samples/google_normal/'+x for x in os.listdir('./samples/google_normal/')]


#Return training/test samples 
def buildData(normal_path, attack_path, test_ratio=0.25):
    X = []
    y = []
    #Get index for attacks
    numAttacks = len(attack_path)
    attack_test_index = np.random.randint(0, numAttacks, int(test_ratio*numAttacks))
    attack_train_index = np.array([i for i in range(numAttacks) if i not in attack_test_index])
    #Get index for normal
    numNormal = len(normal_path)
    normal_test_index = np.random.randint(0, numNormal, int(test_ratio*numNormal))
    normal_train_index = np.array([i for i in range(numNormal) if i not in normal_test_index])

    #Start building data sets
    Xtrain, Xtest, Ytrain, Ytest = [], [], [], []
    for i in attack_train_index:
        '''TRAIN Attacks'''
        mfcc_attack = getAudioFeatures(GOOGLE_ATTACKS[i])
        Xtrain.append(mfcc_attack)
        Ytrain.append(np.ones(mfcc_attack.shape[1])) #attack=1
    for i in attack_test_index:
        '''TEST Attacks'''
        mfcc_attack = getAudioFeatures(GOOGLE_ATTACKS[i])
        Xtest.append(mfcc_attack)
        Ytest.append(np.ones(mfcc_attack.shape[1])) #attack=1   
    for i in normal_train_index:
        '''Train Normal'''
        mfcc_normal = getAudioFeatures(GOOGLE_NORMAL[i])
        Xtrain.append(mfcc_normal)
        Ytrain.append(np.zeros(mfcc_normal.shape[1])) #normal=0
    for i in normal_test_index:
        '''Test Normal'''
        mfcc_normal = getAudioFeatures(GOOGLE_NORMAL[i])
        Xtest.append(mfcc_normal)
        Ytest.append(np.zeros(mfcc_normal.shape[1])) #normal=0    
    Xtrain = np.hstack(Xtrain).T
    Ytrain = np.hstack(Ytrain)
    Xtest = np.hstack(Xtest).T
    Ytest = np.hstack(Ytest)
    return Xtrain, Ytrain, Xtest, Ytest, normal_test_index, attack_test_index

In [186]:
#Get dataset
Xtrain, Ytrain, Xtest, Ytest, normalTestIndex, attackTestIndex = buildData(GOOGLE_NORMAL, GOOGLE_ATTACKS)
print ('Training: ', Xtrain.shape, Ytrain.shape)
print ('Testing: ', Xtest.shape, Ytrain.shape)

Training:  (10434, 4) (10434,)
Testing:  (2153, 4) (10434,)


## 2. Train Model [Logistic Regression]

In [187]:
#Build a model

from sklearn import linear_model

lr = linear_model.LogisticRegression(random_state=42)
print (lr.fit(Xtrain,Ytrain))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [188]:
#Test model all test data points..

Ypredict = lr.predict(Xtest)
print (np.average(Ypredict==Ytest))

0.976776590804


## Let's test on individual audio samples

In [189]:
#Check attack samples
for a in attackTestIndex:
    mfcc = getAudioFeatures(GOOGLE_ATTACKS[a]).T
    ypred = lr.predict(mfcc)
    print (GOOGLE_ATTACKS[a], np.average(ypred))
    
#Check normal samples
for a in normalTestIndex:
    mfcc = getAudioFeatures(GOOGLE_NORMAL[a]).T
    ypred = lr.predict(mfcc)
    print (GOOGLE_NORMAL[a], np.average(ypred))


./samples/google_attacks/Call18002262278.wav 0.982608695652
./samples/google_attacks/CallComcast.wav 0.82722513089
./samples/google_attacks/MuteMyPhone.wav 0.985611510791
./samples/google_attacks/TakeAPicture.wav 0.96
./samples/google_attacks/OpenFacebook.wav 1.0
./samples/google_normal/google_normal_SetAlarm11.wav 0.00396825396825
./samples/google_normal/google_normal_GloryManUnitedYoutube.wav 0.0
./samples/google_normal/google_normal_OpenCNN.wav 0.0047619047619
./samples/google_normal/google_normal_CallComcast.wav 0.00763358778626
./samples/google_normal/google_normal_SetAlarm11.wav 0.00396825396825


In [190]:
#Looking for important features
print(lr.coef_)

[[ 0.03073582  0.11687857  0.20080114  0.11692972]]


In [191]:
print(np.std(Xtrain, 0)*lr.coef_)

[[ 4.48595011  7.28367104  8.88873224  2.94203376]]


In [192]:
print (getAudioFeatures(GOOGLE_ATTACKS[0]))

[[-621.0562037  -618.72309271 -623.98573884 -627.90293786 -624.76406006
  -629.25325916 -630.22940084 -593.73412141 -546.64474461 -514.56903096
  -521.04041629 -557.39696243 -580.60799316 -590.87756941 -514.47234327
  -426.32810082 -391.1975574  -397.31557211 -431.56190488 -497.34663308
  -563.5329149  -559.9999554  -525.16947506 -498.02276508 -490.61677616
  -515.7035119  -549.45318471 -573.09638905 -566.41641537 -549.52134646
  -543.23905255 -544.90686333 -556.07527236 -575.80539625 -592.07814038
  -602.18383846 -611.60031481 -618.48614934 -620.36134405 -621.34329937
  -619.61669938 -619.60915987 -620.30424443 -625.23832573 -623.1899734
  -620.54718572 -620.86965649 -625.87499248 -625.37112655 -623.89729898
  -620.90090157 -622.78788688 -624.26305371 -626.25648919 -629.21876172
  -631.04860539 -627.78768915 -628.40013466 -629.58148359 -628.56837861
  -624.08382846 -624.78773612 -625.69224593 -625.8284176  -625.8602791
  -627.82277958 -624.52574661 -622.47471329 -620.82206797 -619.412

In [193]:
print (getAudioFeatures(GOOGLE_NORMAL[0]))

[[-648.75156566 -647.01827456 -654.18567029 ..., -637.02252225
  -625.31125619 -652.34266086]
 [  98.45524884   99.62522044   93.68285271 ...,   79.63355356
    77.39552442   82.76359742]
 [  21.80004462   17.1020087    16.50995608 ...,   20.65546669
    17.22940471   17.44211898]
 [   6.31829334   -1.08931303   -1.94903106 ...,   -6.54193074
    -8.79254212   -5.07810797]]


In [194]:
# Check new google ones for NORMAL?
a1 = getAudioFeatures('./samples/location_normal.wav').T
ypred = lr.predict(a1)
print (np.average(ypred))

a1 = getAudioFeatures('./samples/statue_normal.wav').T
ypred = lr.predict(a1)
print (np.average(ypred))

0.179611650485
0.351648351648


## Run to plot number of Mel Coefficients vs. Accuracy => average of 100 runs